# 미분양 종합 데이터 시각화

미분양 + 준공 후 미분양 데이터를 통합하여 시각화합니다.

## 데이터 구조
- **미분양수**: 전체 미분양 수량
- **준공_후_미분양수**: 준공 후 미분양 수량 (미분양수에 포함됨)
- **준공_전_미분양수**: 준공 전 미분양 수량 (= 미분양수 - 준공_후_미분양수)

## 시각화 내용
1. 전국 누적 막대그래프 (시점별)
2. 시도별 누적 막대그래프 (특정 시도 선택)
3. 시군구별 누적 막대그래프 (특정 시도의 시군구들)

## 1. 라이브러리 및 데이터 로드

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime

# 데이터 로드 (시점은 문자열로 유지)
df = pd.read_csv('../csv/미분양_종합.csv', encoding='utf-8-sig', dtype={'시점': str})

# 시점을 datetime 형식으로 변환
df['시점_dt'] = pd.to_datetime(df['시점'], format='%Y.%m')

print(f"데이터 로드 완료: {len(df):,}건")
print(f"기간: {df['시점'].min()} ~ {df['시점'].max()}")
print(f"\n컬럼: {', '.join(df.columns.tolist())}")
print(f"\n데이터 샘플:")
df.head(10)

## 2. 전국 미분양 추이 (누적 막대그래프)

In [ ]:
# 전국 데이터 집계 (시점별 합계)
df_nationwide = df.groupby('시점_dt').agg({
    '준공_전_미분양수': 'sum',
    '준공_후_미분양수': 'sum',
    '미분양수': 'sum'
}).reset_index()

df_nationwide = df_nationwide.sort_values('시점_dt')

print(f"전국 데이터: {len(df_nationwide)}개 시점")
print(f"기간: {df_nationwide['시점_dt'].min().strftime('%Y.%m')} ~ {df_nationwide['시점_dt'].max().strftime('%Y.%m')}")

In [ ]:
# 누적 막대그래프 생성
fig = go.Figure()

# 준공 전 미분양 (하단)
fig.add_trace(go.Bar(
    x=df_nationwide['시점_dt'],
    y=df_nationwide['준공_전_미분양수'],
    name='준공 전',
    marker_color='#3498DB',  # 파란색
    hovertemplate='%{x|%Y.%m}<br>준공 전: %{y:,.0f}호<extra></extra>'
))

# 준공 후 미분양 (상단)
fig.add_trace(go.Bar(
    x=df_nationwide['시점_dt'],
    y=df_nationwide['준공_후_미분양수'],
    name='준공 후',
    marker_color='#E74C3C',  # 빨간색
    hovertemplate='%{x|%Y.%m}<br>준공 후: %{y:,.0f}호<extra></extra>'
))

fig.update_layout(
    title='전국 아파트 미분양 추이 (2007-2025)',
    xaxis_title='시점',
    yaxis_title='미분양 수 (호)',
    barmode='stack',  # 누적 모드
    height=600,
    hovermode='x unified',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig.update_xaxes(tickangle=-90, dtick="M12")  # 1년 간격
fig.update_yaxes(tickformat=',')  # y축 천 단위 쉼표 구분

fig.show()

## 3. 시도별 미분양 추이 (누적 막대그래프)

In [ ]:
# 사용자로부터 시도 입력 받기
print("시도 정식 명칭을 입력하세요. (예: 서울특별시, 세종특별자치시, 전북특별자치도)")
print(f"\n사용 가능한 시도:")
for sido in sorted(df['시도'].unique()):
    print(f"  - {sido}")

target_sido = input("\n시도 입력: ")

if target_sido not in df['시도'].unique():
    print(f"\n⚠️ '{target_sido}'는 데이터에 없습니다.")
    print(f"사용 가능한 시도: {', '.join(sorted(df['시도'].unique()))}")
else:
    # 해당 시도의 전체 시군구 데이터 집계
    df_sido = df[df['시도'] == target_sido].groupby('시점_dt').agg({
        '준공_전_미분양수': 'sum',
        '준공_후_미분양수': 'sum',
        '미분양수': 'sum'
    }).reset_index()
    
    df_sido = df_sido.sort_values('시점_dt')
    
    # 누적 막대그래프 생성
    fig = go.Figure()
    
    # 준공 전 미분양 (하단)
    fig.add_trace(go.Bar(
        x=df_sido['시점_dt'],
        y=df_sido['준공_전_미분양수'],
        name='준공 전',
        marker_color='#3498DB',
        hovertemplate='%{x|%Y.%m}<br>준공 전: %{y:,.0f}호<extra></extra>'
    ))
    
    # 준공 후 미분양 (상단)
    fig.add_trace(go.Bar(
        x=df_sido['시점_dt'],
        y=df_sido['준공_후_미분양수'],
        name='준공 후',
        marker_color='#E74C3C',
        hovertemplate='%{x|%Y.%m}<br>준공 후: %{y:,.0f}호<extra></extra>'
    ))
    
    fig.update_layout(
        title=f'{target_sido} 아파트 미분양 추이 (2007-2025)',
        xaxis_title='시점',
        yaxis_title='미분양 수 (호)',
        barmode='stack',
        height=600,
        hovermode='x unified',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    fig.update_xaxes(tickangle=-90, dtick="M12")
    fig.update_yaxes(tickformat=',')  # y축 천 단위 쉼표 구분
    
    fig.show()

## 4. 시군구별 미분양 추이 (누적 막대그래프)

특정 시도의 특정 시군구를 선택하여 해당 시군구의 미분양 추이를 확인합니다.

In [ ]:
# 1단계: 시도 입력
print("시도 정식 명칭을 입력하세요. (예: 서울특별시, 세종특별자치시, 전북특별자치도)")
print(f"\n사용 가능한 시도:")
for sido in sorted(df['시도'].unique()):
    print(f"  - {sido}")

selected_sido = input("\n시도 입력: ")

if selected_sido not in df['시도'].unique():
    print(f"\n⚠️ '{selected_sido}'는 데이터에 없습니다.")
    print(f"사용 가능한 시도: {', '.join(sorted(df['시도'].unique()))}")
else:
    # 해당 시도의 시군구 목록 출력
    sigungu_list = sorted(df[df['시도'] == selected_sido]['시군구'].unique())
    print(f"\n✓ '{selected_sido}' 선택됨")
    print(f"\n{selected_sido}의 시군구 ({len(sigungu_list)}개):")
    for sigungu in sigungu_list:
        print(f"  - {sigungu}")
    
    # 2단계: 시군구 입력
    selected_sigungu = input("\n시군구 입력: ")
    
    if selected_sigungu not in sigungu_list:
        print(f"\n⚠️ '{selected_sigungu}'는 {selected_sido}에 없습니다.")
        print(f"사용 가능한 시군구: {', '.join(sigungu_list)}")
    else:
        print(f"\n✓ '{selected_sigungu}' 선택됨")
        print(f"\n그래프 생성 중...")
        
        # 해당 시군구의 데이터
        df_sigungu = df[(df['시도'] == selected_sido) & (df['시군구'] == selected_sigungu)].copy()
        df_sigungu = df_sigungu.sort_values('시점_dt')
        
        # 누적 막대그래프 생성
        fig = go.Figure()
        
        # 준공 전 미분양 (하단)
        fig.add_trace(go.Bar(
            x=df_sigungu['시점_dt'],
            y=df_sigungu['준공_전_미분양수'],
            name='준공 전',
            marker_color='#3498DB',
            hovertemplate='%{x|%Y.%m}<br>준공 전: %{y:,.0f}호<extra></extra>'
        ))
        
        # 준공 후 미분양 (상단)
        fig.add_trace(go.Bar(
            x=df_sigungu['시점_dt'],
            y=df_sigungu['준공_후_미분양수'],
            name='준공 후',
            marker_color='#E74C3C',
            hovertemplate='%{x|%Y.%m}<br>준공 후: %{y:,.0f}호<extra></extra>'
        ))
        
        fig.update_layout(
            title=f'{selected_sido} {selected_sigungu} 아파트 미분양 추이',
            xaxis_title='시점',
            yaxis_title='미분양 수 (호)',
            barmode='stack',
            height=600,
            hovermode='x unified',
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
            )
        )
        
        fig.update_xaxes(tickangle=-90, dtick="M12")
        fig.update_yaxes(tickformat=',')  # y축 천 단위 쉼표 구분
        
        fig.show()

## 5. 추가 분석

필요에 따라 추가 분석 코드를 작성할 수 있습니다.